In [1]:
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from smotdm.data.text_motion_dataset import TextMotionDataset
from smotdm.models.smotdm import SMOTDM

In [2]:
text_motion_dataset = TextMotionDataset(
    "deps/interx/dataset_med.h5",
)
train_dataloader = DataLoader(
    text_motion_dataset,
    batch_size=1,
    collate_fn=text_motion_dataset.collate_fn,
    shuffle=True,
    num_workers=7,
)

# next(iter(train_dataloader))

In [3]:
model = SMOTDM(
    vae=True,
)

trainer = Trainer(max_epochs=5)

trainer.fit(model, train_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/cogniveon/src/uos/smotdm/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/cogniveon/src/uos/smotdm/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name                   | Type              | Params | Mode 
----------------------------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [26]:
from smotdm.data.collate import collate_x_dict, length_to_mask
from smotdm.models.text_encoder import TextToEmb


text_embeds = TextToEmb("distilbert/distilbert-base-uncased")(
    [
        "Two people walk towards each other. After they meet, the first person hugs the second person around the shoulders, gently patting his/her back with his/her right hand. Meanwhile, the second person puts his/her arms around the first person's waist and pats his/her waist with his/her right hand."
    ]
)
# text_embeds

mask = length_to_mask(text_embeds["length"], device=model.device)
encoded = model.text_encoder(
    {
        "x": text_embeds["x"],
        "mask": mask,
    }
)


dists = encoded.unbind(1)
mu, logvar = dists
latent_vectors = mu
motion = model.motion_decoder(
    {
        "z": latent_vectors,
        "mask": mask,
    }
).squeeze(dim=0)

In [28]:
from smotdm.renderer.matplotlib import SingleMotionRenderer
from smotdm.rifke import feats_to_joints


print(motion.shape)

renderer = SingleMotionRenderer(
    colors=("red", "red", "red", "red", "red"),
)

renderer.render_animation_single(feats_to_joints(motion).detach().cpu().numpy())

torch.Size([68, 166])
